## 1. Introduction

This notebook demonstrates how to manage an EBRAINS OIDC client. Service providers are encouraged to start from this notebook and adapt it to their needs. EBRAINS users need to ask EBRAINS Support to be added to the Collaboratory Group "Service Providers" before they can manage OIDC clients.

**Do not create test OIDC clients in the PROD environment.** Use the INT environment instead.

In [ ]:
import requests
import json

# the API to the OIDC client app
API_PROD = 'https://wiki.ebrains.eu/rest/v1/oidc/clients'
API_INT  = 'https://wiki-int.ebrains.eu/rest/v1/oidc/clients'
'https://iam.humanbrainproject.eu/auth/realms/hbp/clients-registrations/default/tvb-web '
API = API_PROD

# the clientId we use in this notebook
clientId = "tvb-cloud"

## 2. Fetching your Collaboratory access token

In the Collaboratory Lab, users can recover the token with the following command.

Warning: Do not share your token. Do not display your toekn in your notebook otherwise other users with access to the notebook can see the token. Other users do not have access to your variables so storing the token in a variable is not an issue.

In [ ]:
bearer_token = clb_oauth.get_token()

## 3. Creating a new OIDC client

You need the "bearer_token" value from the previous step.

You can create OIDC clients by sending a JSON representation to the OIDC app API endpoint.

In [ ]:
response = requests.post(
    API,
    headers={'Authorization': 'Bearer %s' % bearer_token},
    json={
        "client": 
        {
            "clientId": clientId,
            "name": "The Virtual Brain Web App",
            "description": "TVB Web app in the cloud",
            "rootUrl": "http://tvb-web.apps.tc.humanbrainproject.eu/",
            "baseUrl": "",
            "redirectUris": [
                "https://tvb-web.apps.tc.humanbrainproject.eu/*"
            ],
            "webOrigins": ["+"],
            "bearerOnly": False,
            "consentRequired": True,
            "standardFlowEnabled": True,
            "implicitFlowEnabled": False,
            "directAccessGrantsEnabled": False,
            "publicClient": True, 
            "fullScopeAllowed": True,
            "attributes": {
                "contacts": "lia.domide@codemart.ro; paula.popa@codemart.ro"
            }
        },
        "maintainers": ["paulapopa","ldomide"],
        "featureAuthenticate" : False,
        "accessDeniedToGuests" : True
    })

if response.status_code == requests.codes.created:
    ebrainsClientResponse = response.json()
    print(json.dumps(ebrainsClientResponse, indent=4))
else:
    print("FAILED", response.content)


## 4. Fetching your OIDC client settings

You can check your OIDC client settings simply by providing your clientId. See above.

If you created your OIDC Client before the release of this new API and can't GET it please contact our support, only green line from this excel sheet was moved to the new OIDC API. https://drive.ebrains.eu/lib/ffba272e-e4bd-4be0-8e8c-1f9156c74e47/file/Collaboratory_2_OIDC_clients.xlsx

In [ ]:
response = requests.get(
    f'{API}/{clientId}',
    headers={'Authorization': 'Bearer %s' % bearer_token}
)

if response.status_code == requests.codes.ok:
    ebrainsClientResponse = response.json()
    print(json.dumps(ebrainsClientResponse, indent=4))
else:
    print("FAILED", response.content)

## 5. Modifying your OIDC client settings

We recommend to GET your client first to have a JSON fully filled, to update it and then  use a PUT request to send the updates to the OIDC server and avoid any issues. But theorically, the Keycloak API respects the PUT implementation and you can also just provide a JSON with fields you want to change.

Update your OIDC client settings with a PUT request.

In [ ]:
# Change the description of the OIDC client kg-nexus-role-mapping%20kg-nexus-service-account-mock
#ebrainsClientResponse['client']['optionalClientScopes'] = ["profile", "team", "group",'kg-nexus-role-mapping', 'kg-nexus-service-account-mock']

ebrainsClientResponse['client']["redirectUris"]= [
            "https://tvb-web.apps.tc.humanbrainproject.eu/*",
            "https://tvb-bck.apps.tc.humanbrainproject.eu/*",
            "https://tvb-web.apps.ebrains.eu/*",
            "https://tvb-bck.apps.ebrains.eu/*"
        ]

# Deny access to the OIDC client to Guests users (this is the default, so redundant here)
#ebrainsClientResponse['accessDeniedToGuests'] = True;
   
response = requests.put(
    f'{API}/{clientId}',
    headers={'Authorization': 'Bearer %s' % bearer_token},
    json=ebrainsClientResponse
)

if response.status_code == requests.codes.ok:
    ebrainsClientResponse = response.json()
    print(json.dumps(ebrainsClientResponse, indent=4))
else:
    print("FAILED", response.content)

## 6. Restricting access to you service to specific EBRAINS Groups or Units

By default, any regular EBRAINS account can be authenticated via your OIDC client and therefore have access to your service. You can restrict your service to specific Groups or Units. To do so, set "featureAuthenticate" to true. Once this is done, only users from Groups/Units that you explicitly authorize will have access to your service.

Users can visit their profile page to see Groups and Units they belong to:
- https://wiki.ebrains.eu/bin/view/Identity/ or from the Wiki clicking the profile icon


### 6.1 Set the OIDC client to restrict access

The code below restricts access to the OIDC client's app. After that, no EBRAINS user has access to the app because no Units/Groups have been granted access to it yet.

In [ ]:
ebrainsClientResponse['featureAuthenticate'] = True;

response = requests.put(
    f'{API}/{clientId}',
    headers={'Authorization': 'Bearer %s' % bearer_token},
    json=ebrainsClientResponse
)

if response.status_code == requests.codes.ok:
    ebrainsClientResponse = response.json()
    print(json.dumps(ebrainsClientResponse, indent=4))
else:
    print("FAILED", response.content)

### 6.2 Grant access to a Group

EBRAINS Groups are visible here:
- https://wiki.ebrains.eu/bin/view/Identity/#/groups

**EXAMPLE**: Let's take the group of EBRAINS users authorized to manage OIDC clients:
https://wiki.ebrains.eu/bin/view/Identity/#/groups/app-collaboratory-iam--service-providers

That Group's groupId is "**app-collaboratory-iam--service-providers**". We will grant access to this Group to our app.

In [ ]:
groupId = input("Enter group id, eg : app-collaboratory-iam--service-providers")

In [ ]:
# eg: https://wiki.ebrains.eu/rest/v1/oidc/clients/tutorialOidcApi/groups/app-collaboratory-iam--service-providers
response = requests.put(
    f'{API}/{clientId}/groups/{groupId}',
    headers={'Authorization': 'Bearer %s' % bearer_token}
)

print(response.status_code)

### 6.3 Grant access to a Unit

EBRAINS Units are browsable here:
- https://wiki.ebrains.eu/bin/view/Identity/#/units

The unit path needs to be passed to the API, with path elements separated by colons.

**EXAMPLE:** An example Unit path is "all:projects:hbp:consortium:SGA3"

In [ ]:
unitPath = input("Enter unit path, eg: all:projects:hbp ")

In [ ]:
# eg: https://wiki.ebrains.eu/rest/v1/oidc/clients/tutorialOidcApi/groups/app-collaboratory-iam--service-providers
response = requests.put(
    f'{API}/{clientId}/units/{unitPath}',
    headers={'Authorization': 'Bearer %s' % bearer_token}
)

print(response.status_code)

## 7. Checking who can access your OIDC client's app

You can check that your accesses are well granted by Fetching your OIDC client's settings again from point `4. Fetching your OIDC client settings`

The returned JSON should now list of your Groups/Units granted access.

/!\ : So far you can't grant access directly to specific users, only to Groups and Units.
/!\ : You can't change this list through a modification of your JSON client, you must use the API as shown in point `6.`

    "grantedAccess": {
        "users": [],
        "units": [
            {
                "id": "all:projects:hbp",
                "title": "HBP groups",
                "name": "hbp",
                "description": "All HBP groups",
                "acceptMembershipRequest": false,
                "subUnits": []
            }
        ],
        "groups": [
            {
                "name": "axel-indirect",
                "title": "Axel Indirect",
                "description": "",
                "acceptMembershipRequest": false
            }
        ]
    },

## 8. Remove access to a Group/Unit

It's the same request format as in point `6.` but it's a `delete` call instead of a `put`.

/!\ It will remove a granted access to a Group or Unit. This does not mean that users in this Group or Unit will be specifically denied access to your app; they may have access via another Hroup/Unit that is still in the granted access list. There is no denied access list.

In [ ]:
# eg: https://wiki.ebrains.eu/rest/v1/oidc/clients/tutorialOidcApi/groups/app-collaboratory-iam--service-providers
response = requests.delete(
    f'{API}/{clientId}/groups/{groupId}',
    headers={'Authorization': 'Bearer %s' % bearer_token}
)

print(response.status_code)

In [ ]:
# eg: https://wiki.ebrains.eu/rest/v1/oidc/clients/tutorialOidcApi/groups/app-collaboratory-iam--service-providers
response = requests.delete(
    f'{API}/{clientId}/units/{unitPath}',
    headers={'Authorization': 'Bearer %s' % bearer_token}
)

print(response.status_code)

### 8.1 Check the Group/Unit was removed from the allow list

If you check again your OIDC client settings, the Unit and Group should no longer appear in the JSON response.

In [ ]:
response = requests.get(
    f'{API}/{clientId}',
    headers={'Authorization': 'Bearer %s' % bearer_token}
)

if response.status_code == requests.codes.ok:
    ebrainsClientResponse = response.json()
    print(json.dumps(ebrainsClientResponse, indent=4))
else:
    print(response.content)